In [5]:
import json
import csv
import requests

# Function to generate URLs
def generate_urls():
    urls_with_book_nums = []
    for y in ['6b', '175b']:
        for x in range(268):  # Range is from 0 to 267
            url = f"https://openaipublic.blob.core.windows.net/recursive-book-summ/website/data/booksum_book_trees/{y}/{x}/all.json"
            urls_with_book_nums.append((url, y, x))
    return urls_with_book_nums

# Function to scrape data from a URL
def scrape_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.content)
    else:
        return None


In [6]:
# Specify the desired level
desired_level = 3

# Iterate over each URL and extract data
extracted_data = []
for url, model_size, book_num in generate_urls():
    data = scrape_data(url)
    if data:
        for level, entries in data.items():
            if int(level) == desired_level:
                chunk_index = 0
                for entry in entries:
                    extracted_data.append({
                        "book_text": entry["text"],
                        "depth_3_summary": entry["summary"], 
                        "model_size": model_size,
                        "book_num": book_num, 
                        "document_index": chunk_index
                    })
                    chunk_index += 1
                    
# Write the extracted data to a CSV file
with open('data/extracted_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['book_text', 'depth_3_summary', "model_size", "book_num", "document_index"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for entry in extracted_data:
        writer.writerow(entry)